In [3]:
# import libraries

import openai
import langchain
import pinecone 
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI


In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import os

In [8]:
# lets read the document
import json
from langchain.schema import Document

def read_single_json(file_path):
    """
    Read a single JSON file and convert its content into LangChain Document objects.
    Args:
        file_path (str): Path to the JSON file.
    Returns:
        list: A list of LangChain Document objects.
    """
    documents = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Convert JSON data to Document format
        for key, value in data.items():
            content = f"{key}: {value}" if isinstance(value, str) else f"{key}: {json.dumps(value)}"
            documents.append(Document(page_content=content, metadata={"source": file_path}))

    except Exception as e:
        print(f"Error reading {file_path}: {e}")

    return documents

# Example usage
if __name__ == "__main__":
    file_path = r"C:\Ml Project\AI-Chatbot-with-LLM-VectorDB\changi_jewel_content.json"  # Update with your JSON file path
    docs = read_single_json(file_path)
    for doc in docs:
        print(doc.page_content)


changi_airport: ["useful information for direction passengers", "explore citieswe are flying totoday", "sign up for a changi account to receive the latest updates", "lets give you the best experience possible", "changi airport uses cookies and other innovative tech to deliver an incredible and more personalized experience analytical technologies give us insights on site usage to improve our services marketing technologies from changiairportcom and trusted partners help us advertise our services more relevantly if you choose tokeep them off the basic functionality may degrade your experience preferences can be changed at any time with future effect for more information see ourprivacy policy"]
jewel_changi_airport: ["discover a wondrous world where nature meets retail", "discover a wondrous world where nature meets retail", "the firstofitskind play attractions at jewel promise endless fun for the young and young at heart", "indulge in the wide array of dining options at jewel", "a worldc

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_data(docs, chunk_size=800, chunk_overlap=50):
    """
    Chunk documents into smaller pieces using RecursiveCharacterTextSplitter.
    Args:
        docs (list): List of Document objects to be chunked.
        chunk_size (int): Maximum size of each chunk.
        chunk_overlap (int): Number of overlapping characters between chunks.
    Returns:
        list: List of chunked Document objects.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunked_docs = text_splitter.split_documents(docs)
    return chunked_docs

if __name__ == "__main__":
    # Assuming `docs` is a list of Document objects
    chunked_docs = chunk_data(docs)
    for chunk in chunked_docs:
        print(chunk.page_content)



changi_airport: ["useful information for direction passengers", "explore citieswe are flying totoday", "sign up for a changi account to receive the latest updates", "lets give you the best experience possible", "changi airport uses cookies and other innovative tech to deliver an incredible and more personalized experience analytical technologies give us insights on site usage to improve our services marketing technologies from changiairportcom and trusted partners help us advertise our services more relevantly if you choose tokeep them off the basic functionality may degrade your experience preferences can be changed at any time with future effect for more information see ourprivacy policy"]
jewel_changi_airport: ["discover a wondrous world where nature meets retail", "discover a wondrous world where nature meets retail", "the firstofitskind play attractions at jewel promise endless fun for the young and young at heart", "indulge in the wide array of dining options at jewel", "a worldc

In [12]:
# Import required libraries
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Example documents
docs = [
    Document(page_content="This is a long document about Changi Airport and its attractions. " 
                          "The Jewel is a major highlight, including the Rain Vortex and Canopy Park. "
                          "Dining options are plentiful, offering both local and international cuisines.", 
             metadata={"source": "example.json"})
]

# Function to chunk documents
def chunk_data(docs, chunk_size=100, chunk_overlap=20):
    """
    Chunk documents into smaller pieces using RecursiveCharacterTextSplitter.
    Args:
        docs (list): List of Document objects to be chunked.
        chunk_size (int): Maximum size of each chunk.
        chunk_overlap (int): Number of overlapping characters between chunks.
    Returns:
        list: List of chunked Document objects.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return text_splitter.split_documents(docs)

# Chunk the data
chunked_documents = chunk_data(docs)

# Check the number of chunks
print(f"Number of chunks: {len(chunked_documents)}")

# Print content of chunks
for i, chunk in enumerate(chunked_documents):
    print(f"Chunk {i + 1}: {chunk.page_content}")


Number of chunks: 3
Chunk 1: This is a long document about Changi Airport and its attractions. The Jewel is a major highlight,
Chunk 2: a major highlight, including the Rain Vortex and Canopy Park. Dining options are plentiful,
Chunk 3: are plentiful, offering both local and international cuisines.


In [14]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
text = "Changi Airport is one of the best airports in the world."
embedding = model.encode(text)
print(embedding)

c:\Ml Project\AI-Chatbot-with-LLM-VectorDB\venv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\newth\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


[ 6.01984896e-02  4.27370183e-02  6.03869110e-02  4.34192568e-02
  5.23741879e-02 -3.48874852e-02  6.34245574e-02 -9.64083243e-03
 -3.06542069e-02  2.50709411e-02  1.02230720e-02 -3.06353550e-02
  4.35768031e-02  6.56639338e-02  1.17072435e-02  1.48841543e-02
 -1.36543987e-02 -7.12105334e-02 -7.10620433e-02 -8.85656625e-02
 -4.82185790e-03 -4.95686894e-03 -2.31575556e-02  1.46760596e-02
 -7.25708306e-02 -5.41962571e-02 -8.67303647e-03  6.15465231e-02
  3.81670371e-02 -6.35394081e-03 -7.39156529e-02  9.90081504e-02
 -4.27832194e-02  4.46158350e-02 -3.34210275e-03 -5.39374596e-04
 -4.21519503e-02 -4.34518270e-02  6.62737489e-02 -3.11782956e-02
 -1.57658830e-02  5.47460578e-02  5.84814548e-02 -9.56310406e-02
  2.37366911e-02 -6.89463764e-02  2.05923561e-02 -1.45807698e-01
  1.97384376e-02  5.45701198e-02  7.93529227e-02  3.54916789e-02
  7.63163269e-02  5.44120781e-02 -1.88720636e-02  8.14747624e-03
 -7.47628286e-02 -9.29688513e-02  3.07394024e-02 -6.75935596e-02
 -1.11291371e-02  1.92684

In [16]:

print(len(embedding))  # This will print the length of the embedding vector

384
